In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Caminho base dos dados
BASE_PATH = '/content/drive/MyDrive/dados/'

# Arquivos a serem carregados
files = [
    'order_products.csv', 'orders.csv', 'products.csv', 'prices.csv', 'discounts.csv',
    'special_prices.csv', 'users.csv', 'companies.csv', 'products_approval.csv', 'homologated_products.csv'
]

# Carrega os datasets em um dict
datasets = {f.split('.')[0]: pd.read_csv(BASE_PATH + f) for f in files}

# Função para exibir informações gerais e valores nulos
def explorar(df, nome):
    print(f'\n--- {nome} ---')
    print(df.info())
    display(df.describe())
    display(df.isnull().sum())

# Explorar dados iniciais
for name, df in datasets.items():
    explorar(df, name)

# Colunas a remover por dataset
cols_to_drop = {
    'order_products': ['delivery_date'],
    'orders': ['subsidiary_id', 'seller_id'],
    'products': ['name', 'description', 'image_url'],
    'prices': ['expiration_date'],
    'users': ['encrypted_password', 'reset_password_token', 'reset_password_sent_at', 'remember_created_at', 'resume'],
    'products_approval': ['name', 'description']
}

# Remover colunas desnecessárias
for name, cols in cols_to_drop.items():
    datasets[name].drop(columns=cols, inplace=True, errors='ignore')

# Separar colunas de data e hora
def processar_datas(df, cols):
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').dt.floor('s')
            df[f'{col}_date'] = df[col].dt.floor('D')
            df[f'{col}_hour'] = df[col] - df[col].dt.floor('D')
            df.drop(columns=col, inplace=True)
    return df

# Definir colunas de datas para cada dataset
date_columns = {
    'order_products': ['price_date', 'created_at', 'updated_at'],
    'orders': ['order_date', 'created_at', 'updated_at'],
    'products': ['created_at', 'updated_at'],
    'prices': ['created_at', 'updated_at'],
    'discounts': ['created_at', 'updated_at'],
    'special_prices': ['created_at', 'updated_at'],
    'users': ['created_at', 'updated_at'],
    'companies': ['created_at', 'updated_at'],
    'products_approval': ['created_at', 'updated_at'],
    'homologated_products': ['created_at', 'updated_at'],
}

# Aplicar processamento de datas
for name, cols in date_columns.items():
    datasets[name] = processar_datas(datasets[name], cols)

# Renomear colunas
rename_maps = {
    'order_products': {
        'order_id': 'id_pedido',
        'supplier_product_id': 'id_produto_fornecedor',
        'price_id': 'id_preco',
        'price_value': 'valor_do_preco',
        'quantity': 'quantidade',
        'total_price': 'preco_total',
        'price_date_date': 'data_preco',
        'price_date_hour': 'hora_preco',
        'created_at_date': 'data_criacao',
        'created_at_hour': 'hora_criacao',
        'updated_at_date': 'data_da_atualizacao',
        'updated_at_hour': 'hora_da_atualizacao'
    },
    'orders': {
        'company_id': 'id_empresa',
        'user_id': 'id_usuario',
        'order_value': 'valor_do_pedido',
        'supplier_company_id': 'id_empresa_fornecedora',
        'order_date_date': 'data_pedido',
        'order_date_hour': 'hora_pedido',
        'created_at_date': 'data_criacao',
        'created_at_hour': 'hora_criacao',
        'updated_at_date': 'data_da_atualizacao',
        'updated_at_hour': 'hora_da_atualizacao'
    },
    # Adicione outros dicionários de renomeação conforme necessário...
}

for name, mapping in rename_maps.items():
    datasets[name].rename(columns=mapping, inplace=True)

# Tratar valores ausentes específicos
if 'cpf' in datasets['users'].columns:
    datasets['users']['cpf'] = datasets['users']['cpf'].fillna(0)

# One-Hot Encoding em colunas categóricas específicas
one_hot_cols = {
    'orders': ['status'],
    'prices': ['tipo_preco'],
    'companies': ['tipo_empresa'],
    'products_approval': ['status_atual']
}

def aplicar_one_hot(df, cols):
    df = pd.get_dummies(df, columns=cols)
    for col in df.columns:
        if any(col.startswith(prefix + '_') for prefix in cols):
            df[col] = df[col].astype(int)
    return df

for name, cols in one_hot_cols.items():
    datasets[name] = aplicar_one_hot(datasets[name], cols)

# Imputar valores numéricos ausentes com mediana
imputer = SimpleImputer(strategy='median')
for name, cols in {
    'prices': ['id_produto_fornecedor'],
    'homologated_products': ['valor_ultima_compra', 'quantidade_ultima_compra']
}.items():
    datasets[name][cols] = imputer.fit_transform(datasets[name][cols])

# Ajustar tipos para merge (exemplos)
int_columns = {
    'order_products': ['valor_do_preco', 'preco_total'],
    'orders': ['valor_do_pedido'],
    'prices': ['id_produto_fornecedor'],
    'homologated_products': ['valor_ultima_compra', 'quantidade_ultima_compra']
}
for name, cols in int_columns.items():
    for col in cols:
        if col in datasets[name].columns:
            datasets[name][col] = datasets[name][col].astype(int)

# Converter datas em homologated_products e preencher NaT
datasets['homologated_products']['data_ultima_compra'] = pd.to_datetime(
    datasets['homologated_products'].get('data_ultima_compra', pd.NaT), errors='coerce'
).fillna(pd.Timestamp('1900-01-01'))

# Mostrar amostra final dos datasets processados
for name, df in datasets.items():
    print(f'\nDataset: {name}')
    display(df.head())


ModuleNotFoundError: No module named 'pandas'